In [1]:
# -*- coding: utf-8 -*-
#pip install requests beautifulsoup4 lxml selenium

In [8]:
import warnings
warnings.filterwarnings("ignore")

### 저장 경로 및 크롤링 사이트 지정 / html 구조 분석 

In [32]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import os
import time

#저장 경로 
save_path = "./data/smart_factory_crawling_result.csv"
# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://www.dbpia.co.kr/search/topSearch?startCount=0&collection=ALL&range=A&searchField=ALL&sort=RANK&query=%EC%8A%A4%EB%A7%88%ED%8A%B8+%ED%8C%A9%ED%86%A0%EB%A6%AC&srchOption=*&includeAr=false#none"

#내용이 자바스크립트로 실행되기 때문에, 스크립트가 실행된 후 크롤링 시도
driver_path = os.path.join("./chromedriver_win32/", "chromedriver")
driver = webdriver.Chrome(driver_path, options=webdriver.ChromeOptions().add_argument("headless"))
driver.get(source_url)

# 더보기 클릭하여 모든 목록 불러오기
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
searchCount = soup.find("span", "searchCount").text #검색 결과 숫자로 반복할 횟수 정함
count = re.sub(r'[^0-9]', '', searchCount)
count = int(count) // 10 + 1 

SCROLL_PAUSE_TIME = 1.5 
for index in range(count):
    try :
        driver.find_element_by_xpath("//div[@class='listFoot']/div[@class='paginate']/a[@class='viewMore']").click() 
        time.sleep(SCROLL_PAUSE_TIME) 
    except :  
        break 
    
# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
html = driver.page_source
driver.close()
soup = BeautifulSoup(html, 'lxml')

div_contRight = soup.find("div", { "class" : "contRight" })

div_listBody = div_contRight.find("div", { "class" : "listBody" })
li_item = div_listBody.find_all("li", { "class" : "item" })
print("item size : " + str(len(li_item)))

item size : 564


### 데이처 추출하고 저장

In [ ]:
import pandas as pd #데이터 프레임으로 저장 
import random
import numpy as np

# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비합니다.
columns = ['Type', 'Title', 'Author', 'Year', 'Abstract', 'Link']
df = pd.DataFrame(columns=columns)

#초록 추출
def get_abstract(url):
    abstractText = ""    
    if link : 
        driver_path = os.path.join("./chromedriver_win32/", "chromedriver")
        driver = webdriver.Chrome(driver_path, options=webdriver.ChromeOptions().add_argument("headless"))
        driver.get(link)

        html = driver.page_source
        soup2 = BeautifulSoup(html, 'lxml')
        
        abstractDiv = soup2.find("div", { "class" : "abstractWrap" })
        articleP = abstractDiv.find_all("p", { "class" : "article" })
        
        if articleP :
            for article in articleP :
                while True :
                    if article.span :
                        article.span.decompose() 
                    else :
                        break;
                abstractText = abstractText + article.text
        driver.close()
    
    return abstractText

page_url_base = "https://www.dbpia.co.kr"
for index in range(len(li_item)):
    try : #타입 
        data_type = li_item[index].find("ul", { "class" : "type" }).find("li", { "class" : "data" }).text
    except: 
        data_type = ""
        
    try : #제목
        title = li_item[index].find("div", { "class" : "titWrap" }).find("h5").text
    except: 
        title = ""
       
    try : #저자
        author = li_item[index].find("li", {"class" : "author"}).text
    except: 
        author = ""
       
    try : #발행연도
        year = li_item[index].find("li", {"class" : "date"}).text
    except: 
        year = ""
        
    try : #링크, 초록
        titUrl = li_item[index].find("a").get('href')
        link = page_url_base + titUrl
        abstract = get_abstract(link)
    except Exception as e :
        print('오류 발생~!.', e)
        link = ""
        abstract = ""  
       
    row = [data_type, title, author, year, abstract, link]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)
    
    #속도 조절
    rand_value = np.random.randint(1, 5)
    time.sleep(rand_value)
       
df.to_csv(save_path, encoding='utf-8')
